# Lezione 8 - Toy experiment e integrazione Monte-Carlo

## Simulazione di un esperimento con toy experiment
- Spesso si utilizzano sequenze di numeri pseudo-casuali per simulare il comportamento statistico di un esperimento di misura oppure nei metodi di integrazione numerica
- Nella pratica, si ripete un esperimento un certo numero di volte, ottenendo una collezione di esperimenti. Ciascuno di questi esperimenti avrà le proprie statistiche (media, varianza, ...), da queste si possono ricavare statistiche sulla collezione di tutti gli esperimenti (media delle medie, deviazione standard delle medie, ...)
- Questo risulta utile, per esempio per trovare la distribuzione di uno stimatore, determinare se è biased, verificarne la consistenza etc.
- La simulazione di un esperimento di misura è detta "toy experiment"

### Esempio: distribuzione delle medie di un campione
```Python
from stats import Stats
from random_methods import gen_unif

# Variabili
x_min: float = -10.0
x_max: float = 10.0
N: int = 100 # Numero di eventi per esperimento
N_toys: int = 1000 # Numero di toy experiment
means: list[float] = [] # Collezione delle medie degli esperimenti

# Loop sui toy experiment
for i in range(N_toys):
    sample = gen_unif(x_min, x_max, N)
    toy_stats = Stats(sample)
    means.append(toy_stats.mean())
```

Alla fine, la lista `means` conterrà le medie associate ad ognuno dei toy experiment. E' possibile rappresentare `means` in un istogramma per ottenere la distribuzione delle medie, che per il TCL sarà una gaussiana per $ N \to +\infty $

### Confronto con la deviazione standard della media
La deviazione standard della media per ciascuno dei toy experiment, essendo l'incertezza associata alla media delle misure, deve necessariamente coincidere con la deviazione standard della distribuzione delle medie.

## Integrazione con numeri pseudo-casuali
- Si possono impiegare sequenze di numeri pseudo-casuali per calcolare l'area sottesa dal grafico di una funzione
- I metodi di integrazione numerica che si servono di numeri pseudo-casuali sono detti "metodi Monte-Carlo"

Per semplificare, si fanno le seguenti assunzioni:
- Si considerano funzioni positive e continue definite su un intervallo $ [a, b] $ compatto e connesso

### Metodo hit-or-miss
L'algoritmo hit-or-miss consiste nel generare coppie di numeri casuali nel rettangolo $ [a, b] \times [0, y_{max}] $ similmente a quanto già visto nel metodo TAC per la generazione di numeri pseudo-casuali. Chiamando $ n_{hit} $ il numero di punti che cadono sotto il grafico della funzione, $ N $ il numero totale di eventi e $ A = y_{max} \cdot (b - a) $, si avrà

$$ \lim_{N \to +\infty}{\frac{n_{hit}}{N}} = \frac{1}{A} \cdot \int_{a}^{b}{g(x) dx} \implies \int_{a}^{b}{g(x) dx} = \lim_{N \to +\infty}{\frac{n_{hit}}{N}} \cdot A $$

Quindi in uno scenario reale, in cui $ N $ è finito ma abbastanza grande
$$ \int_{a}^{b}{g(x) dx} \simeq \frac{n_{hit}}{N} \cdot A = I $$

Perciò $ I $ è uno stimatore del valore dell'integrale, e come tale avrà un proprio valore di aspettazione e una propria varianza. Dal momento che $ n_{hit} $ segue una distribuzione binomiale, mentre $ A $ e $ N $ sono numeri noti senza incertezza

$$ E[I] = E\left[A \cdot \frac{n_{hit}}{N}\right] = \frac{A}{N} \cdot E[n_{hit}] = \frac{A}{N} \cdot Np = A \cdot p = A \cdot \frac{n_{hit}}{N} $$

$$ V[I] = V\left[A \cdot \frac{n_{hit}}{N}\right] = \frac{A^2}{N^2} \cdot V[n_{hit}] = \frac{A^2}{N^2} \cdot Np(1 - p) = \frac{A^2}{N} \cdot p(1 - p) $$

dove ovviamente $ p = \frac{n_{hit}}{N} $

### Metodo crude Monte-Carlo
Il metodo crude Monte-Carlo sfrutta le proprietà del valore di aspettazione di una funzione: dato un insieme di numeri pseudo-casuali $ x_{i} $ generati secondo una distribuzione di probabilità **uniforme** $ f(x) $ definita su $ [a, b] $, il valore di aspettazione di $ g(x) $ risulta essere

$$ E[g(x)] = \int_{a}^{b}{g(x)f(x) dx} = \frac{1}{b - a} \cdot \int_{a}^{b}{g(x) dx} \implies \int_{a}^{b}{g(x) dx} = (b - a) \cdot E[g(x)] $$

Tenendo conto anche dell'errore 

$$ \int_{a}^{b}{g(x) dx} = (b - a) \cdot \left(E[g(x)] \pm \sqrt{\frac{V[g(x)]}{N}}\right) $$

dove $ E[g(x)] $ può essere stimato calcolando la media dei valori $ g(x_{i}) $, mentre $ V[g(x)] $ può essere calcolata a partire dalla varianza campionaria dei valori $ g(x_{i}) $